In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
%cd "/content/gdrive/MyDrive/..." # change to your path	

/content/gdrive/.shortcut-targets-by-id/1syPiNVQUgh60gmvbP12NyL0c-k21dKeU/Introduction to Natural Language Processing/Lab3


https://www.gradio.app/guides/real-time-speech-recognition

In [11]:
! pip install -U sentence-transformers

In [12]:
! pip install gradio -q

In [13]:
!pip install langchain_community

#### Prompts

In [14]:
# PROMPTS 0-4-shot

some_comments_from_dev1 = '''It's better to say a moment like that could truly ignite her love for the game rather than putting a bit of a damper on it.
I went to a destination wedding being the only single person. Promised to never put myself in that situation again.
He died 4 days later of dehydration
Like this just cuz of the [NAME] rhymes background raps...but dude your [NAME] is sick against [NAME]
Lol dream on buddy. You’ve had enough attention today. Actually learn what your talking about helps a lot. Sorry your stuck in free roam smokin crack
As an anesthesia resident this made me blow air out my nose at an accelerated rate for several seconds. Take your damn upvote you bastard.
1-2-3-4 I declare a thumb war! Dangit [NAME], you win again! Ok you get to stab me again :(
'''
some_comments_from_dev2 = '''I can't believe how fast the time flies, it feels like just yesterday we were starting this project.
Honestly, I thought it would be much harder, but turns out, I actually enjoyed it quite a bit.
Why do bad things always happen to good people? It just doesn't seem fair at all.
Wow, your dedication to your craft is truly inspiring. Keep up the amazing work!
I'm so frustrated with this situation. No matter what I do, nothing seems to go right.
Lol, you really think that will work? Good luck with that, you'll need it!
I'm excited for the weekend! Got any fun plans?'''

some_comments_from_dev3 = '''There's something really satisfying about finishing a tough workout. Endorphins are no joke!
I just had the best meal ever, I can't stop thinking about that dessert!
Why does every season finale leave me in tears? I can't handle these emotional rollercoasters.
Your speech at the event was so moving, you really know how to captivate an audience.
I feel so burnt out from work. I need a vacation ASAP.
Haha, that joke was hilarious! I'm still laughing!
Can't wait to see you this weekend, it's been way too long since we hung out.'''

some_comments_from_dev4 = '''It's amazing how a simple act of kindness can make someone's day.
I've never felt this excited about a project before, the possibilities are endless!
Why do I always misplace my keys at the worst possible times?
Your attention to detail is incredible, it's what sets your work apart.
Feeling a bit under the weather today, hoping it's just a 24-hour bug.
That movie was so bad, I couldn't stop laughing at how ridiculous it was.
Looking forward to our next adventure together, it's going to be epic!'''

summary_from_dev1 = "Comments include personal anecdotes, humorous exchanges, and playful banter. They reflect a mix of light-heartedness, frustration, and shared experiences, ranging from a wedding experience, a humorous medical resident's reaction, to playful competition and a casual rap mention."
summary_from_dev2 = "The comments express a mix of nostalgia, unexpected enjoyment, frustration with unfairness and challenging situations, admiration for dedication, skepticism, and excitement for the weekend."
summary_from_dev3 = "The comments express satisfaction from personal achievements, enjoyment of food, emotional reactions to TV shows, appreciation for a moving speech, work burnout, humor, and excitement about reuniting with a friend."
summary_from_dev4 = "The comments reflect a range of emotions and experiences, including appreciation for kindness and attention to detail, excitement about new projects, minor frustrations, humor in unexpected places, and anticipation for future adventures, all combined with a sense of optimism and camaraderie."

prompt_zero_shot = """
    You are a summarizer bot.
    Your task is to given a group of comments you should return a general summary of the group of comments.
    You must not summarize each comment individually, you should summarize them as a whole.
    Describe general emotions.
    Do not provide additional explanations or notes, just the summary.
    Do not use more than 60 words.
    The group of comments is inside <<<>>>.

    <<<{comments}>>>
"""

prompt_one_shot = """
    You are a summarizer bot.
    Your task is to given a group of comments you should return a general summary of the group of comments.
    You must not summarize each comment individually, you should summarize them as a whole.
    Describe general emotions.
    Do not provide additional explanations or notes, just the summary.
    Do not use more than 60 words.
    The group of comments is inside <<<>>>.

    ###
    Here are some examples:

    Group of comments: {some_comments_from_dev1}
    Summary: {summary_from_dev1}

    ###

    <<<{comments}>>>
"""

prompt_two_shot = """
    You are a summarizer bot.
    Your task is to given a group of comments you should return a general summary of the group of comments.
    You must not summarize each comment individually, you should summarize them as a whole.
    Describe general emotions.
    Do not provide additional explanations or notes, just the summary.
    Do not use more than 60 words.
    The group of comments is inside <<<>>>.

    ###
    Here are some examples:

    Group of comments: {some_comments_from_dev1}
    Summary: {summary_from_dev1}
    Group of comments: {some_comments_from_dev2}
    Summary: {summary_from_dev2}

    ###

    <<<{comments}>>>
"""

prompt_three_shot = """
    You are a summarizer bot.
    Your task is to given a group of comments you should return a general summary of the group of comments.
    You must not summarize each comment individually, you should summarize them as a whole.
    Describe general emotions.
    Do not provide additional explanations or notes, just the summary.
    Do not use more than 60 words.
    The group of comments is inside <<<>>>.

    ###
    Here are some examples:

    Group of comments: {some_comments_from_dev1}
    Summary: {summary_from_dev1}
    Group of comments: {some_comments_from_dev2}
    Summary: {summary_from_dev2}
    Group of comments: {some_comments_from_dev3}
    Summary: {summary_from_dev3}

    ###

    <<<{comments}>>>
"""

prompt_four_shot = """
    You are a summarizer bot.
    Your task is to given a group of comments you should return a general summary of the group of comments.
    You must not summarize each comment individually, you should summarize them as a whole.
    Describe general emotions.
    Do not provide additional explanations or notes, just the summary.
    Do not use more than 60 words.
    The group of comments is inside <<<>>>.

    ###
    Here are some examples:

    Group of comments: {some_comments_from_dev1}
    Summary: {summary_from_dev1}
    Group of comments: {some_comments_from_dev2}
    Summary: {summary_from_dev2}
    Group of comments: {some_comments_from_dev3}
    Summary: {summary_from_dev3}
    Group of comments: {some_comments_from_dev4}
    Summary: {summary_from_dev4}

    ###

    <<<{comments}>>>
"""

#### Functions

In [15]:
import os
from dotenv import load_dotenv, find_dotenv
from huggingface_hub import login, notebook_login
a = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HUGGINGFACEHUB_API_TOKEN']
os.environ['TRANSFORMERS_CACHE'] = './cache/'
your_token = '...' # your token here
login(token=your_token, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [22]:
import gradio as gr
from transformers import pipeline
from scipy import spatial
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch
import os
from langchain_community.llms import HuggingFaceHub
from langchain.schema import (
    HumanMessage,
    SystemMessage,
    AIMessage
)
from langchain_community.chat_models.huggingface import ChatHuggingFace

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")
sentence_transformer_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# your classification model shall be loaded here
pipe = pipeline("text-classification", model="bert_large_uncased_goemotions4", return_all_scores=True, device=-1 if device.type=="cpu" else 0)
label_names = open("data/emotions.txt", "r").read().splitlines()


# your vectors for similar comments to be retrieved shall be loaded here. You can store them in comment_vectors.npy file
train_df = pd.read_csv("data/train.tsv", sep = "\t", names = ["text", "labels", "id"])
dev_df = pd.read_csv("data/dev.tsv", sep = "\t", names = ["text", "labels", "id"])

all_sentences = train_df['text'].tolist() + dev_df['text'].tolist()
all_labels = train_df['labels'].tolist() + dev_df['labels'].tolist()

if not os.path.exists("comment_vectors.npy"):
    embedding_array = sentence_transformer_model.encode(all_sentences, batch_size=64, show_progress_bar=False, device=device)
    np.save("comment_vectors.npy", embedding_array) # to save embeddings

# your LLM model for summarization shall be loaded here
llm = HuggingFaceHub(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

chat_model = ChatHuggingFace(llm=llm)

def predict_emotions(comment_text_, classes_threshold=0.3):
    """Here we predict emotions using fine-tuned transformer model.
    """

    all_classes_scored = pipe(comment_text_)
    predicted_emotions = []
    for pred in all_classes_scored[0]:
        if pred["score"] > classes_threshold:
            predicted_emotions.append((label_names[int(pred["label"])]))

    return predicted_emotions


def get_similar_comments(comment_text_, top_n_comments=10, similarity_threshold=0.5):
    """Here we retrieve comments from dev/train set which are similar to a given comment text.
    """
    # Load your pre-calculated sentence-embedding for the training and dev set (at least 10000 examples)
    embedding_array = np.load("comment_vectors.npy") # to load them
    # Calculate sentence-embedding for the comment_text_
    comment_embedding = sentence_transformer_model.encode([comment_text_])[0]
    # Use spatial.distance.cdist() from Word embeddings.ipynb to calculate similarities between comments
    comments_similarities = 1 - spatial.distance.cdist([comment_embedding], embedding_array, metric = "cosine")[0]
    # Select top_n most similar commentsm and return them in a list that shall include corresponding emotions and similarity scores
    top_n_indices = np.argsort(comments_similarities)[::-1][:top_n_comments]

    most_similar_comments = [
      {"text": all_sentences[i],
      "emotions": [label_names[int(label)] for label in all_labels[i].split(",")],
      "similarity_score": comments_similarities[i]}
      for i in top_n_indices
      if comments_similarities[i] >= similarity_threshold  # Only include comments above the threshold
]

    return most_similar_comments


def create_prompt(comments, shots):
    if shots == 0:
        return prompt_zero_shot.format(comments=comments)
    elif shots == 1:
        return prompt_one_shot.format(comments=comments, some_comments_from_dev1=some_comments_from_dev1, summary_from_dev1=summary_from_dev1)
    elif shots == 2:
        return prompt_two_shot.format(comments=comments, some_comments_from_dev1=some_comments_from_dev1, summary_from_dev1=summary_from_dev1, some_comments_from_dev2=some_comments_from_dev2, summary_from_dev2=summary_from_dev2)
    elif shots == 3:
        return prompt_three_shot.format(comments=comments, some_comments_from_dev1=some_comments_from_dev1, summary_from_dev1=summary_from_dev1, some_comments_from_dev2=some_comments_from_dev2, summary_from_dev2=summary_from_dev2, some_comments_from_dev3=some_comments_from_dev3, summary_from_dev3=summary_from_dev3)
    elif shots == 4:
        return prompt_four_shot.format(comments=comments, some_comments_from_dev1=some_comments_from_dev1, summary_from_dev1=summary_from_dev1, some_comments_from_dev2=some_comments_from_dev2, summary_from_dev2=summary_from_dev2, some_comments_from_dev3=some_comments_from_dev3, summary_from_dev3=summary_from_dev3, some_comments_from_dev4=some_comments_from_dev4, summary_from_dev4=summary_from_dev4)


def get_summary(top_n_similar_comments_):
    messages = [
        HumanMessage(content="Hello"),
        AIMessage(content="How can I help you?"),
        HumanMessage(
            content=create_prompt(top_n_similar_comments_, shots=0)
        ),
    ]
    res = chat_model.invoke(messages)
    summary = res.content.split("INST]")[-1].strip()
    return summary


def analyze_comment(stream, new_chunk, input_text, similarity_threshold=0.5, classes_threshold=0.3):
    if new_chunk:
        sr, y = new_chunk
        y = y.astype(np.float32)
        y /= np.max(np.abs(y))

        if stream is not None:
            stream = np.concatenate([stream, y])
        else:
            stream = y

    # if input_text is filled with a text, we won't apply audio transcriber
    comment_text = None
    if not input_text:
        comment_text = transcriber({"sampling_rate": sr, "raw": stream})["text"]
    else:
        comment_text = input_text

    emotions = predict_emotions(comment_text, classes_threshold=classes_threshold)
    emotions_as_text = ", ".join(emotions)
    similar_comments = get_similar_comments(comment_text, top_n_comments=10, similarity_threshold=similarity_threshold)
    generated_summary = get_summary(similar_comments)

    similar_comments_as_text = "\n\n".join([("Text: %s\nEmotions: %s\nSimilarity score: %.2lf") %
     (comment["text"], ", ".join(comment["emotions"]), comment["similarity_score"]) for comment in similar_comments])

    return stream, comment_text, emotions_as_text, similar_comments_as_text, generated_summary


print(analyze_comment(stream=None, new_chunk=None, input_text="What a wonderful day!"))

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


(None, 'What a wonderful day!', 'excitement, joy', "Text: Oh what a day! What a lovely day!\nEmotions: joy\nSimilarity score: 0.80\n\nText: Have a wonderful day.\nEmotions: excitement, neutral\nSimilarity score: 0.76\n\nText: Have a lovely day.\nEmotions: love\nSimilarity score: 0.64\n\nText: I hope you have a wonderful cake day!\nEmotions: optimism\nSimilarity score: 0.59\n\nText: Have a nice day\nEmotions: caring\nSimilarity score: 0.58\n\nText: Goodmorning, I wish you all a wonderful day.\nEmotions: optimism\nSimilarity score: 0.57\n\nText: What a wonderful world\nEmotions: admiration\nSimilarity score: 0.57\n\nText: Like I said please have a great day !\nEmotions: caring\nSimilarity score: 0.55\n\nText: I was having a good day. We were ALL having a good day.\nEmotions: admiration, joy\nSimilarity score: 0.54\n\nText: Lovely day we're having.\nEmotions: excitement\nSimilarity score: 0.53", 'Commenters express joy, excitement, love, and optimism, wishing others a wonderful day.')


In [17]:
def get_summary(top_n_similar_comments_):
    messages = [
        HumanMessage(content="Hello"),
        AIMessage(content="How can I help you?"),
        HumanMessage(
            content=create_prompt(top_n_similar_comments_, shots=4)
        ),
    ]
    res = chat_model.invoke(messages)
    summary = res.content.split("INST]")[-1].strip()
    return summary

In [23]:
# Type "What a wonderful day!" in input_text and press "Submit" to see the dummy outcome

demo = gr.Interface(
    analyze_comment,
    inputs=[
        "state",
        gr.Audio(sources=["microphone"], streaming=False),
        "text",
        gr.Slider(0, 1, step=0.01, label="Similarity Threshold", value=0.5),
        gr.Slider(0, 1, step=0.01, label="Classes Threshold", value=0.3)

    ],
    outputs = ["state", gr.Textbox(label="Test comment"),
               gr.Textbox(label="Predicted emotions"),
               gr.Textbox(label="Similar comments"),
               gr.Textbox(label="Summary")],
    live=False,
)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eeb8e10352fc430027.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
